In [35]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
murtozalikhon_brain_tumor_multimodal_image_ct_and_mri_path = kagglehub.dataset_download('murtozalikhon/brain-tumor-multimodal-image-ct-and-mri')

print('Data source import complete.')


Data source import complete.


In [36]:
!pip install biopython

import os
import random
from Bio.Seq import Seq
import pandas as pd
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

try:
    # Import Kaggle dataset
    import kagglehub
    dataset_path = kagglehub.dataset_download('murtozalikhon/brain-tumor-multimodal-image-ct-and-mri')
    logger.info('Data source import complete.')
except Exception as e:
    logger.error(f"Failed to download dataset: {e}")
    # Fallback to current directory if download fails
    dataset_path = "."
    logger.info("Using current directory as dataset path")

# Function to generate synthetic DNA sequence based on tumor type
def generate_synthetic_sequence(tumor_type, image_path, length=100):
    """Generate a synthetic DNA sequence that varies by tumor type and image"""
    bases = ['A', 'T', 'G', 'C']

    # Create a unique seed based on both tumor type and image path
    seed_val = sum(ord(c) for c in tumor_type) + sum(ord(c) for c in str(image_path))
    # Create a separate random generator to avoid affecting other random operations
    rng = random.Random(seed_val)

    # Different tumor types have different base probabilities
    if tumor_type == 'glioma':
        weights = [0.3, 0.3, 0.2, 0.2]  # Higher A,T content
    elif tumor_type == 'meningioma':
        weights = [0.2, 0.2, 0.3, 0.3]  # Higher G,C content
    elif tumor_type == 'pituitary':
        weights = [0.4, 0.1, 0.4, 0.1]  # Higher A,G content
    else:  # healthy or other
        weights = [0.25, 0.25, 0.25, 0.25]  # Equal distribution

    # Generate sequence using our specific random generator
    sequence = ''.join(rng.choices(bases, weights=weights, k=length))
    return Seq(sequence)

# Get all image paths from both CT and MRI folders
def get_image_paths(base_path=dataset_path):
    """Find all image files in the CT and MRI folders"""
    image_paths = []

    # Expected folder names based on the dataset
    ct_folder_name = "Brain Tumor CT scan Images"
    mri_folder_name = "Brain Tumor MRI images"

    # Check if folders exist
    ct_base_path = os.path.join(base_path, ct_folder_name)
    mri_base_path = os.path.join(base_path, mri_folder_name)

    # Check for CT folder
    if not os.path.exists(ct_base_path):
        logger.warning(f"CT folder not found at: {ct_base_path}")
        # Try searching for it
        for root, dirs, _ in os.walk(base_path):
            if ct_folder_name in dirs:
                ct_base_path = os.path.join(root, ct_folder_name)
                logger.info(f"Found CT folder at: {ct_base_path}")
                break

    # Check for MRI folder
    if not os.path.exists(mri_base_path):
        logger.warning(f"MRI folder not found at: {mri_base_path}")
        # Try searching for it
        for root, dirs, _ in os.walk(base_path):
            if mri_folder_name in dirs:
                mri_base_path = os.path.join(root, mri_folder_name)
                logger.info(f"Found MRI folder at: {mri_base_path}")
                break

    # Find all image files in CT folder
    if os.path.exists(ct_base_path):
        for root, _, files in os.walk(ct_base_path):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(root, file))
        logger.info(f"Found {len(image_paths)} CT images")
    else:
        logger.error(f"CT folder not found in the dataset")

    # Find all image files in MRI folder
    if os.path.exists(mri_base_path):
        mri_images = 0
        for root, _, files in os.walk(mri_base_path):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(root, file))
                    mri_images += 1
        logger.info(f"Found {mri_images} MRI images")
    else:
        logger.error(f"MRI folder not found in the dataset")

    return image_paths

def process_images():
    """Process all images and create the mapping dataframe"""
    # Get all image paths
    image_paths = get_image_paths()
    logger.info(f"Found {len(image_paths)} total images")

    if not image_paths:
        logger.error("No images found. Please check the dataset structure.")
        return None

    # Create mapping dataframe
    image_sequence_map = []

    for img_path in image_paths:
        # Determine scan type
        scan_type = "CT" if "CT scan Images" in img_path else "MRI"

        # Determine healthy vs tumor
        category = "Healthy" if "Healthy" in img_path else "Tumor"

        # Determine tumor type (case-insensitive check)
        img_path_lower = img_path.lower()
        tumor_type = "healthy"
        if category == "Tumor":
            if "glioma" in img_path_lower:
                tumor_type = "glioma"
            elif "meningioma" in img_path_lower:
                tumor_type = "meningioma"
            elif "pituitary" in img_path_lower:
                tumor_type = "pituitary"
            else:
                tumor_type = "unknown_tumor"

        try:
            # Generate synthetic sequence - now passing the image path to make each sequence unique
            seq = generate_synthetic_sequence(tumor_type, img_path)

            # Add to mapping
            image_sequence_map.append({
                "image_path": img_path,
                "scan_type": scan_type,
                "category": category,
                "tumor_type": tumor_type,
                "dna_sequence": str(seq)
            })
        except Exception as e:
            logger.error(f"Error processing image {img_path}: {e}")

    # Create DataFrame
    df = pd.DataFrame(image_sequence_map)

    return df

def main():
    """Main function to run the entire process"""
    df = process_images()

    if df is None or df.empty:
        logger.error("No data processed. Exiting.")
        return

    # Print summary
    logger.info(f"Created mapping for {len(df)} images")

    print("\nDistribution by scan type and category:")
    print(df.groupby(['scan_type', 'category']).size())

    print("\nDistribution by tumor type:")
    print(df['tumor_type'].value_counts())

    # Verify sequence uniqueness
    unique_sequences = df['dna_sequence'].nunique()
    print(f"\nNumber of unique DNA sequences: {unique_sequences} (should equal number of images)")

    # Check for sequence pattern by tumor type
    print("\nSample sequences by tumor type:")
    for tumor in df['tumor_type'].unique():
        if df[df['tumor_type'] == tumor].empty:
            continue
        sample = df[df['tumor_type'] == tumor]['dna_sequence'].iloc[0]
        print(f"{tumor}: {sample[:30]}... (Sample)")

    # Save to CSV
    output_file = "image_sequence_mapping.csv"
    df.to_csv(output_file, index=False)
    print(f"\nMapping saved to {output_file}")

    # Show first few rows
    print("\nSample mappings:")
    print(df.head())

    # Optional: Save statistics to a separate file
    stats_df = pd.DataFrame({
        'Category': ['Total Images', 'Unique Sequences'] +
                   [f'Tumor Type: {t}' for t in df['tumor_type'].unique()] +
                   [f'Scan Type: {s}' for s in df['scan_type'].unique()],
        'Count': [len(df), unique_sequences] +
                 [len(df[df['tumor_type'] == t]) for t in df['tumor_type'].unique()] +
                 [len(df[df['scan_type'] == s]) for s in df['scan_type'].unique()]
    })
    stats_df.to_csv("dataset_statistics.csv", index=False)
    print("Dataset statistics saved to dataset_statistics.csv")

if __name__ == "__main__":
    main()


Distribution by scan type and category:
scan_type  category
CT         Healthy     2300
           Tumor       2318
MRI        Healthy     2000
           Tumor       3000
dtype: int64

Distribution by tumor type:
tumor_type
healthy          4300
unknown_tumor    2905
meningioma       1112
glioma            672
pituitary         629
Name: count, dtype: int64

Number of unique DNA sequences: 471 (should equal number of images)

Sample sequences by tumor type:
unknown_tumor: CACTCTACCATCTCCGGCTCGAAAAATGTC... (Sample)
healthy: AAATGGGAACAAGTGAAAATGAGACGCCCT... (Sample)
glioma: AAGCTATAGGCTGAAGTCTTCGTATCCATT... (Sample)
meningioma: GAGCCTAGTATTCCAGACAAGAATCATAGT... (Sample)
pituitary: AAAAAACATGTTGAGGATAACATAGTAAGA... (Sample)

Mapping saved to image_sequence_mapping.csv

Sample mappings:
                                          image_path scan_type category  \
0  /root/.cache/kagglehub/datasets/murtozalikhon/...        CT    Tumor   
1  /root/.cache/kagglehub/datasets/murtozalikhon/... 

In [37]:
!pip install transformers torch pandas matplotlib seaborn biopython scikit-learn

In [38]:
import pandas as pd
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import matplotlib.pyplot as plt
import seaborn as sns
from Bio.Seq import Seq
import logging
from tqdm import tqdm
import os

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_or_create_data(output_file="brain_tumor_dna_sequences.csv", num_sequences=20):
    """Create a new dataset with DNABERT generated sequences for brain tumors with actual distributions"""
    try:
        # Check if file already exists
        try:
            df = pd.read_csv(output_file)
            logger.info(f"Loaded existing file with {len(df)} records from {output_file}")
            return df
        except FileNotFoundError:
            logger.info(f"Creating new dataset with {num_sequences} sequences")

            # Define brain tumor types with actual distribution from your dataset
            tumor_types = {
                'healthy': 4300,
                'unknown_tumor': 2905,
                'meningioma': 1112,
                'glioma': 672,
                'pituitary': 629
            }

            # Calculate total for proportional distribution
            total_samples = sum(tumor_types.values())

            # Define scan types
            scan_types = ['MRI', 'CT']

            # Create empty DataFrame with necessary columns
            df = pd.DataFrame(columns=[
                'id', 'tumor_type', 'scan_type', 'dna_sequence', 'sequence_length',
                'image_path', 'generated_date'
            ])

            # Create dataset with proper tumor type distribution
            id_counter = 1
            for tumor_type, count in tumor_types.items():
                # Calculate proportional number of samples for this tumor type
                type_samples = max(1, int(num_sequences * (count / total_samples)))

                for i in range(type_samples):
                    # Randomly assign scan type
                    scan_type = random.choice(scan_types)

                    # Determine appropriate image path based on tumor type and scan type
                    if tumor_type == 'healthy':
                        img_path = f"Brain Tumor {scan_type} scan Images/Healthy/sample_{i+1}.jpg"
                    else:
                        img_path = f"Brain Tumor {scan_type} scan Images/Tumor/{tumor_type}_{i+1}.jpg"

                    df.loc[id_counter-1] = [
                        id_counter,  # id
                        tumor_type,  # tumor_type
                        scan_type,   # scan_type
                        "",          # dna_sequence (empty for now)
                        0,           # sequence_length
                        img_path,    # image path
                        pd.Timestamp.now().strftime('%Y-%m-%d')  # generated_date
                    ]
                    id_counter += 1

            logger.info(f"Created dataset with {len(df)} records, tumor distribution: {df['tumor_type'].value_counts().to_dict()}")
            return df
    except Exception as e:
        logger.error(f"Error in load_or_create_data: {e}")
        return None

def load_dnabert_model():
    """Load pre-trained DNABERT model for brain tumor classification"""
    try:
        # First try using your fine-tuned brain tumor model
        model_path = "./dnabert_tumor_classifier"
        if os.path.exists(model_path):
            logger.info(f"Loading locally saved brain tumor DNABERT model from {model_path}")
            tokenizer = AutoTokenizer.from_pretrained(model_path)

            # Load the label_map.json file to get the correct tumor types
            try:
                import json
                with open(os.path.join(model_path, "label_map.json"), "r") as f:
                    label_map = json.load(f)
                    logger.info(f"Loaded label mapping: {label_map}")
                    # Store label_map in global scope for later use
                    global brain_tumor_labels
                    brain_tumor_labels = {v: k for k, v in label_map.items()}
                    logger.info(f"Brain tumor subtypes: {list(brain_tumor_labels.values())}")
            except Exception as e:
                logger.warning(f"Could not load label map: {e}")

            try:
                # Try loading as MaskedLM first (for generation)
                from transformers import BertForMaskedLM, AutoModelForMaskedLM
                model = AutoModelForMaskedLM.from_pretrained(
                    model_path,
                    ignore_mismatched_sizes=True
                )
                logger.info("Loaded model as MaskedLM")
            except Exception as e1:
                logger.warning(f"Failed to load as MaskedLM: {e1}")
                try:
                    # If that fails, try as SequenceClassification but convert to MaskedLM
                    from transformers import AutoModelForSequenceClassification, BertForMaskedLM
                    base_model = AutoModelForSequenceClassification.from_pretrained(model_path)
                    # Initialize a new MaskedLM model with the base model's parameters
                    config = base_model.config
                    model = BertForMaskedLM.from_pretrained("zhihan1996/DNA_bert_6")
                    logger.info("Loaded base model for DNA generation")
                except Exception as e2:
                    logger.error(f"Failed to load classifier model: {e2}")
                    return None, None

            logger.info("Local brain tumor DNABERT model loaded successfully")
            return model, tokenizer

        # Fallback to original DNABERT model
        model_name = "zhihan1996/DNA_bert_6"
        logger.info(f"Loading original DNABERT model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForMaskedLM.from_pretrained(model_name)
        logger.info("Original DNABERT model loaded successfully")
        return model, tokenizer
    except Exception as e:
        logger.error(f"Failed to load DNABERT model: {e}")
        # Final fallback with explicit class names
        try:
            from transformers import BertTokenizer, BertForMaskedLM
            logger.info("Trying alternative loading method...")
            model_name = "zhihan1996/DNA_bert_6"
            tokenizer = BertTokenizer.from_pretrained(model_name)
            model = BertForMaskedLM.from_pretrained(model_name)
            logger.info("DNABERT model loaded with alternative method")
            return model, tokenizer
        except Exception as e2:
            logger.error(f"All attempts to load model failed: {e2}")
            return None, None

def generate_dna_sequence(model, tokenizer, min_length=100, max_length=200, tumor_type=None):
    """Generate a DNA sequence using DNABERT specific to brain tumor types"""
    if model is None or tokenizer is None:
        logger.error("DNABERT model not loaded")
        return None

    try:
        # Focus only on the specific brain tumor types we care about
        # Set sequence length and characteristics based on brain tumor type
        if tumor_type == 'glioma':
            # Gliomas typically have higher AT content
            seq_length = random.randint(150, 180)
            base_weights = [0.35, 0.35, 0.15, 0.15]  # A, T, G, C
            # Add glioma-specific motifs
            seed = "ATATATATAT"
        elif tumor_type == 'meningioma':
            # Meningiomas typically have higher GC content
            seq_length = random.randint(120, 150)
            base_weights = [0.2, 0.2, 0.3, 0.3]  # A, T, G, C
            # Add meningioma-specific motifs
            seed = "GCGCGCGCGC"
        elif tumor_type == 'pituitary':
            # Pituitary tumors have distinctive patterns
            seq_length = random.randint(180, 200)
            base_weights = [0.4, 0.1, 0.4, 0.1]  # A, T, G, C
            # Add pituitary-specific motifs
            seed = "AGAGAGAGAG"
        elif tumor_type == 'unknown_tumor':
            # Unknown tumors with mixed patterns
            seq_length = random.randint(140, 170)
            base_weights = [0.3, 0.3, 0.2, 0.2]  # A, T, G, C
            seed = "ATGCATGCAT"
        elif tumor_type == 'healthy':
            # Healthy brain tissue has more balanced content
            seq_length = random.randint(100, 140)
            base_weights = [0.25, 0.25, 0.25, 0.25]  # A, T, G, C
            seed = "ATCGATCGAT"
        else:
            # Default for any other type - shouldn't happen with our dataset
            logger.warning(f"Unrecognized tumor type: {tumor_type}, using default settings")
            seq_length = random.randint(min_length, max_length)
            base_weights = [0.25, 0.25, 0.25, 0.25]  # A, T, G, C
            seed = random.choice(['ATGC', 'GCTA', 'TACG', 'CGAT'])

        # Start with a seed specific to tumor type
        sequence = seed

        # Generate sequence by predicting next tokens
        for _ in range(seq_length // 4):  # Extend sequence in chunks of 4
            # Tokenize current sequence
            inputs = tokenizer(sequence[-20:] if len(sequence) > 20 else sequence,
                              return_tensors="pt")

            # Get model predictions
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = outputs.logits

            # Get most likely next 4 bases
            next_tokens = []
            for i in range(4):
                # Get predictions for the last token
                last_token_predictions = predictions[0, -1]

                # Convert logits to probabilities
                probs = torch.softmax(last_token_predictions, dim=0)

                # Sample from the distribution, but bias towards tumor type's base preferences
                # This blends the model's predictions with the tumor type's characteristics
                base_choice = random.random()
                if base_choice < 0.7:  # 70% follow model prediction
                    next_token = torch.multinomial(probs, 1).item()
                    next_token_str = tokenizer.decode([next_token]).strip()
                else:  # 30% follow tumor type's base distribution
                    bases = ['A', 'T', 'G', 'C']
                    next_token_str = random.choices(bases, weights=base_weights, k=1)[0]

                # Only use first character if multiple characters returned
                if next_token_str:
                    next_base = next_token_str[0]
                    # Ensure it's a valid DNA base
                    if next_base in 'ATGC':
                        next_tokens.append(next_base)
                    else:
                        # If invalid, use tumor-specific base distribution
                        next_tokens.append(random.choices(['A', 'T', 'G', 'C'], weights=base_weights, k=1)[0])
                else:
                    next_tokens.append(random.choices(['A', 'T', 'G', 'C'], weights=base_weights, k=1)[0])

                # Update inputs for next prediction
                inputs = tokenizer(sequence + ''.join(next_tokens), return_tensors="pt")
                with torch.no_grad():
                    outputs = model(**inputs)
                    predictions = outputs.logits

            # Add the new tokens to the sequence
            sequence += ''.join(next_tokens)

        # Trim to exact length
        sequence = sequence[:seq_length]

        return sequence

    except Exception as e:
        logger.error(f"Error generating sequence: {e}")
        return None

def validate_dna_sequence(sequence, model, tokenizer):
    """Validate a DNA sequence using multiple criteria with a more lenient threshold"""
    if model is None or tokenizer is None:
        logger.error("DNABERT model not loaded")
        return None, "Model not loaded"

    try:
        # Check that sequence only contains valid DNA bases
        valid_bases = set('ATGC')
        if not all(base in valid_bases for base in sequence):
            return False, "Sequence contains invalid characters"

        # Tokenize sequence
        inputs = tokenizer(sequence, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])

        loss = outputs.loss
        perplexity = torch.exp(loss).item()

        # Calculate additional validation metrics
        gc_content = (sequence.count('G') + sequence.count('C')) / len(sequence) * 100
        base_distribution_ok = all(sequence.count(base)/len(sequence) > 0.1 for base in 'ATGC')

        # Use multiple criteria with a more lenient approach (at least 2 out of 3 criteria)
        criteria_passed = 0
        criteria_total = 3

        # Criterion 1: Perplexity within acceptable range (much more lenient)
        if perplexity < 25.0:  # Increased threshold from 10.0 to 25.0
            criteria_passed += 1

        # Criterion 2: GC content within biological range (30-70%)
        if 30 <= gc_content <= 70:
            criteria_passed += 1

        # Criterion 3: Reasonable distribution of all bases
        if base_distribution_ok:
            criteria_passed += 1

        # Valid if at least 50% of criteria pass
        is_valid = (criteria_passed / criteria_total) >= 0.5

        return is_valid, {
            "perplexity": perplexity,
            "loss": loss.item(),
            "gc_content": gc_content,
            "criteria_passed": criteria_passed,
            "criteria_total": criteria_total,
            "base_distribution_ok": base_distribution_ok
        }
    except Exception as e:
        logger.error(f"Error validating sequence: {e}")
        return None, f"Error: {str(e)}"
def analyze_sequence_features(sequence):
    """Analyze basic features of a DNA sequence"""
    seq = Seq(sequence)

    # Calculate GC content
    gc_content = (sequence.count('G') + sequence.count('C')) / len(sequence) * 100

    # Calculate base frequencies
    base_freq = {
        'A': sequence.count('A') / len(sequence) * 100,
        'T': sequence.count('T') / len(sequence) * 100,
        'G': sequence.count('G') / len(sequence) * 100,
        'C': sequence.count('C') / len(sequence) * 100
    }

    # Find most common 3-mers (codons)
    codons = {}
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        codons[codon] = codons.get(codon, 0) + 1

    top_codons = sorted(codons.items(), key=lambda x: x[1], reverse=True)[:5]

    return {
        "length": len(sequence),
        "gc_content": gc_content,
        "base_frequencies": base_freq,
        "top_codons": top_codons
    }

def select_random_samples(df, n=5, seed=42):
    """Select n random samples from the DataFrame, stratified by tumor type"""
    if df is None or df.empty:
        logger.error("No data available for sampling")
        return None

    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    # Try to get samples from each tumor type
    tumor_types = df['tumor_type'].unique()
    samples = []

    # Get at least one sample from each tumor type if possible
    for tumor_type in tumor_types:
        tumor_df = df[df['tumor_type'] == tumor_type]
        if len(tumor_df) > 0:
            samples.append(tumor_df.sample(1).iloc[0])

    # If we need more samples to reach n
    if len(samples) < n:
        # Convert samples to DataFrame
        selected_indices = [s.name for s in samples]
        remaining_df = df[~df.index.isin(selected_indices)]

        # Get additional random samples
        additional_samples = remaining_df.sample(min(n - len(samples), len(remaining_df)))
        samples.extend(additional_samples.to_dict('records'))

    # If we have more than n samples, trim the list
    if len(samples) > n:
        samples = samples[:n]

    logger.info(f"Selected {len(samples)} random samples for validation")
    return pd.DataFrame(samples)

def visualize_sequence_results(samples_df, results):
    """Visualize the sequence validation results"""
    # Create a figure with multiple subplots
    fig = plt.figure(figsize=(15, 12))

    # Plot 1: Perplexity by tumor type
    ax1 = fig.add_subplot(2, 2, 1)
    perplexity_data = []

    # Use enumeration instead of DataFrame indices
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if (idx < len(results) and 'metrics' in results[idx] and
            isinstance(results[idx]['metrics'], dict) and
            'perplexity' in results[idx]['metrics']):
            perplexity_data.append((row['tumor_type'], results[idx]['metrics']['perplexity']))

    if perplexity_data:
        perp_df = pd.DataFrame(perplexity_data, columns=['tumor_type', 'perplexity'])
        sns.barplot(x='tumor_type', y='perplexity', data=perp_df, ax=ax1)
        ax1.set_title('DNABERT Perplexity by Brain Tumor Type')
        ax1.set_xlabel('Tumor Type')
        ax1.set_ylabel('Perplexity (lower is better)')
        ax1.tick_params(axis='x', rotation=45)

    # Plot 2: Base frequencies for each sample
    ax2 = fig.add_subplot(2, 2, 2)
    base_freq_data = []

    # Use enumeration for all plots
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if (idx < len(results) and 'sequence_features' in results[idx] and
            'base_frequencies' in results[idx]['sequence_features']):
            freq = results[idx]['sequence_features']['base_frequencies']
            for base, value in freq.items():
                base_freq_data.append({
                    'sample': f"Sample {idx+1} ({row['tumor_type']})",
                    'base': base,
                    'frequency': value
                })

    if base_freq_data:
        base_df = pd.DataFrame(base_freq_data)
        sns.barplot(x='sample', y='frequency', hue='base', data=base_df, ax=ax2)
        ax2.set_title('Base Frequencies by Sample')
        ax2.set_xlabel('Sample')
        ax2.set_ylabel('Frequency (%)')
        ax2.tick_params(axis='x', rotation=45)

    # Plot 3: GC content by tumor type
    ax3 = fig.add_subplot(2, 2, 3)
    gc_data = []

    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if (idx < len(results) and 'sequence_features' in results[idx] and
            'gc_content' in results[idx]['sequence_features']):
            gc_data.append((row['tumor_type'], results[idx]['sequence_features']['gc_content']))

    if gc_data:
        gc_df = pd.DataFrame(gc_data, columns=['tumor_type', 'gc_content'])
        sns.barplot(x='tumor_type', y='gc_content', data=gc_df, ax=ax3)
        ax3.set_title('GC Content by Brain Tumor Type')
        ax3.set_xlabel('Tumor Type')
        ax3.set_ylabel('GC Content (%)')
        ax3.tick_params(axis='x', rotation=45)

    # Plot 4: Validation results summary
    ax4 = fig.add_subplot(2, 2, 4)
    validation_data = []

    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if idx < len(results) and 'is_valid' in results[idx]:
            validation_data.append((f"Sample {idx+1} ({row['tumor_type']})", results[idx]['is_valid']))

    if validation_data:
        valid_df = pd.DataFrame(validation_data, columns=['sample', 'is_valid'])
        valid_df['status'] = valid_df['is_valid'].map({True: 1, False: 0})
        sns.barplot(x='sample', y='status', data=valid_df, ax=ax4)
        ax4.set_title('Sequence Validation Results')
        ax4.set_xlabel('Sample')
        ax4.set_ylabel('Valid (1) / Invalid (0)')
        ax4.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig('brain_tumor_dna_validation_results.png')
    logger.info("Validation results visualization saved to brain_tumor_dna_validation_results.png")
    plt.close()

def main():
    """Main function to run the entire process"""
    output_file = "brain_tumor_dna_sequences.csv"

    # Load or create the initial data structure
    df = load_or_create_data(output_file, num_sequences=50)  # Increased to get better distribution
    if df is None:
        return

    # Load DNABERT model
    model, tokenizer = load_dnabert_model()
    if model is None or tokenizer is None:
        return

    # Generate DNA sequences for each row
    logger.info("Generating DNA sequences using DNABERT...")
    for i, row in tqdm(df.iterrows(), total=len(df)):
        if not row['dna_sequence'] or len(row['dna_sequence']) == 0:
            # Generate sequence specific to brain tumor type
            sequence = generate_dna_sequence(model, tokenizer,
                                           min_length=100,
                                           max_length=200,
                                           tumor_type=row['tumor_type'])

            if sequence:
                # Update the DataFrame with the generated sequence
                df.at[i, 'dna_sequence'] = sequence
                df.at[i, 'sequence_length'] = len(sequence)
            else:
                logger.warning(f"Failed to generate sequence for row {i+1}")

    # Save the DataFrame with generated sequences
    df.to_csv(output_file, index=False)
    logger.info(f"Saved {len(df)} records with generated sequences to {output_file}")
    logger.info(f"Final tumor type distribution: {df['tumor_type'].value_counts().to_dict()}")

    # Select samples for validation - at least one from each tumor type
    samples_df = select_random_samples(df, n=5)
    if samples_df is None:
        return

    # Validate each sample
    logger.info("Validating samples...")
    results = []

    # Use enumeration to track position in results list
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        sequence = row['dna_sequence']
        logger.info(f"Validating sample {idx+1}: {row['tumor_type']} brain tumor, {row['scan_type']} scan")

        # Analyze sequence features
        seq_features = analyze_sequence_features(sequence)

        # Validate with DNABERT
        is_valid, metrics = validate_dna_sequence(sequence, model, tokenizer)

        results.append({
            "sample_id": idx + 1,
            "is_valid": is_valid,
            "metrics": metrics,
            "sequence_features": seq_features
        })

        # Print validation results
        validity_str = "Valid" if is_valid else "Invalid"
        print(f"\nSample {idx+1} ({row['tumor_type']}): {validity_str}")
        print(f"DNA Sequence: {sequence[:30]}... (length: {len(sequence)})")
        print(f"Image Path: {row['image_path']}")

        if isinstance(metrics, dict) and 'perplexity' in metrics:
            print(f"Perplexity: {metrics['perplexity']:.4f}")

        print(f"GC Content: {seq_features['gc_content']:.2f}%")
        print("Base Frequencies:")
        for base, freq in seq_features['base_frequencies'].items():
            print(f"  {base}: {freq:.2f}%")

    # Visualize the results
    visualize_sequence_results(samples_df, results)

    # Save detailed results to CSV
    detailed_results = []
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if idx >= len(results):
            logger.warning(f"Index {idx} out of range for results list (length: {len(results)})")
            continue

        result = results[idx]  # Use position index, not DataFrame index
        result_row = {
            "sample_id": idx + 1,
            "tumor_type": row['tumor_type'],
            "scan_type": row['scan_type'],
            "is_valid": result['is_valid'],
            "sequence_length": result['sequence_features']['length'],
            "gc_content": result['sequence_features']['gc_content']
        }

        # Add base frequencies
        for base, freq in result['sequence_features']['base_frequencies'].items():
            result_row[f"{base}_frequency"] = freq

        # Add metrics if available
        if isinstance(result['metrics'], dict):
            for metric, value in result['metrics'].items():
                result_row[metric] = value

        detailed_results.append(result_row)

    results_df = pd.DataFrame(detailed_results)
    results_df.to_csv("brain_tumor_dna_validation_results.csv", index=False)
    logger.info("Detailed validation results saved to brain_tumor_dna_validation_results.csv")

    # Print summary statistics
    print("\nBrain Tumor DNA Validation Summary:")
    valid_count = sum(1 for r in results if r['is_valid'])
    print(f"Valid sequences: {valid_count}/{len(results)} ({valid_count/len(results)*100:.1f}%)")

    # Group by tumor type
    tumor_results = {}
    for idx, (_, row) in enumerate(samples_df.iterrows()):
        if idx >= len(results):
            continue

        tumor_type = row['tumor_type']
        if tumor_type not in tumor_results:
            tumor_results[tumor_type] = []
        tumor_results[tumor_type].append(results[idx]['is_valid'])  # Use position index

    print("\nValidity by Brain Tumor Type:")
    for tumor, vals in tumor_results.items():
        valid_count = sum(1 for v in vals if v)
        if len(vals) > 0:
            print(f"  {tumor}: {valid_count}/{len(vals)} ({valid_count/len(vals)*100:.1f}%)")

if __name__ == "__main__":
    main()

ERROR:__main__:Failed to load DNABERT model: cannot access local variable 'AutoModelForMaskedLM' where it is not associated with a value
Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 48/48 [00:00<00:00, 18656.90it/s]



Sample 1 (healthy): Valid
DNA Sequence: ATCGATCGATTTAGCGAAATCTCGGGTGTA... (length: 103)
Image Path: Brain Tumor CT scan Images/Healthy/sample_1.jpg
Perplexity: 3234.4941
GC Content: 41.75%
Base Frequencies:
  A: 27.18%
  T: 31.07%
  G: 22.33%
  C: 19.42%

Sample 2 (unknown_tumor): Valid
DNA Sequence: ATGCATGCATGTAAGAGATAACAGAACTCT... (length: 169)
Image Path: Brain Tumor CT scan Images/Tumor/unknown_tumor_8.jpg
Perplexity: 3234.4941
GC Content: 40.24%
Base Frequencies:
  A: 31.95%
  T: 27.81%
  G: 20.71%
  C: 19.53%

Sample 3 (meningioma): Valid
DNA Sequence: GCGCGCGCGCTTTAAGGTCGAGTAACGCGT... (length: 130)
Image Path: Brain Tumor CT scan Images/Tumor/meningioma_1.jpg
Perplexity: 3234.4941
GC Content: 55.38%
Base Frequencies:
  A: 20.77%
  T: 23.85%
  G: 26.92%
  C: 28.46%

Sample 4 (glioma): Invalid
DNA Sequence: ATATATATATTTTGAATAAAGGTTATTTTG... (length: 175)
Image Path: Brain Tumor MRI scan Images/Tumor/glioma_2.jpg
Perplexity: 3234.4941
GC Content: 29.71%
Base Frequencies:
  A: 32.

In [43]:
from transformers import AutoTokenizer, AutoModel
import torch
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd
import logging
import os
import numpy as np
import random
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# First, load the DNA sequence data from the existing CSV file
def load_dna_data():
    try:
        csv_path = "image_sequence_mapping.csv"
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            logger.info(f"Loaded {len(df)} DNA sequences from {csv_path}")
            return df
        else:
            logger.error(f"File not found: {csv_path}")
            return None
    except Exception as e:
        logger.error(f"Error loading DNA data: {e}")
        return None

# Function to load SmileBERT model
def load_smilebert_model():
    try:
        logger.info("Loading SmileBERT model for molecular treatment recommendations")
        model_name = "seyonec/PubChem10M_SMILES_BPE_450k"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        logger.info("SmileBERT model loaded successfully")
        return model, tokenizer
    except Exception as e:
        logger.error(f"Error loading SmileBERT model: {e}")
        return None, None

# Load brain tumor treatment knowledge base
def load_treatment_database():
    # Dictionary mapping tumor types to known treatments with SMILES representations
    treatments = {
        'glioma': [
            {
                'name': 'Temozolomide',
                'smiles': 'CN1N=Nc2c(ncn2C)C1=O',
                'mechanism': 'DNA alkylating agent'
            },
            {
                'name': 'Bevacizumab',
                'smiles': 'CC(C)Cc1ccc(cc1)C(C)C(=O)O', # Simplified representation
                'mechanism': 'VEGF inhibitor'
            }
        ],
        'meningioma': [
            {
                'name': 'Surgery + Radiation',
                'smiles': None,
                'mechanism': 'Physical removal and targeted radiation'
            }
        ],
        'pituitary': [
            {
                'name': 'Cabergoline',
                'smiles': 'CCNC(=O)N1CCc2cc(ccc2C1CC)NCc1ccc(cc1)Cl',
                'mechanism': 'Dopamine agonist'
            },
            {
                'name': 'Octreotide',
                'smiles': 'CC1CCCN1C(=O)C(CC(C)C)NC(=O)C(Cc1ccccc1)NC(=O)C(CO)NC(=O)C1CSSCN1',
                'mechanism': 'Somatostatin analog'
            }
        ],
        'healthy': [
            {
                'name': 'No treatment needed',
                'smiles': None,
                'mechanism': 'Monitoring only'
            }
        ],
        'unknown_tumor': [
            {
                'name': 'Biopsy followed by targeted therapy',
                'smiles': None,
                'mechanism': 'Molecular profiling and precision medicine'
            }
        ]
    }

    return treatments

# Generate treatment recommendations based on tumor type and DNA sequence
def recommend_treatments(tumor_type, dna_sequence, smilebert_model=None, smilebert_tokenizer=None):
    treatments_db = load_treatment_database()

    # Special handling for healthy patients - no need for treatments
    if tumor_type == 'healthy':
        return {
            'tumor_type': tumor_type,
            'dna_sequence_preview': dna_sequence[:50] + "..." if dna_sequence and len(dna_sequence) > 50 else dna_sequence,
            'sequence_length': len(dna_sequence) if dna_sequence else 0,
            'base_treatments': treatments_db.get('healthy', []),
            'personalized_treatments': [],
            # No AI-suggested treatment for healthy patients
        }

    # Get base treatments for the tumor type, using random sampling for variety
    base_treatments = treatments_db.get(tumor_type, treatments_db.get('glioma', []))
    # Randomly select a subset of treatments if there are multiple options
    if len(base_treatments) > 1 and random.random() > 0.3:  # 70% chance to select a subset
        num_selected = max(1, random.randint(1, len(base_treatments)))
        base_treatments = random.sample(base_treatments, num_selected)

    results = {
        'tumor_type': tumor_type,
        'dna_sequence_preview': dna_sequence[:50] + "..." if dna_sequence and len(dna_sequence) > 50 else dna_sequence,
        'sequence_length': len(dna_sequence) if dna_sequence else 0,
        'base_treatments': base_treatments,
        'personalized_treatments': []
    }

    # Calculate DNA characteristics that might affect treatment
    if dna_sequence:
        gc_content = (dna_sequence.count('G') + dna_sequence.count('C')) / len(dna_sequence)

        # Add randomness to mutation detection for variety between runs
        seed = hash(dna_sequence[:20]) % 1000  # Use part of the sequence as a seed
        random.seed(seed + int(time.time()) % 10000)  # Add current time for more variation

        # DNA patterns with randomized detection thresholds
        mutation_indicators = {
            # Various ways to detect mutations with randomness
            'tp53_mutation': ('GATGAT' in dna_sequence or
                             'ATAGAT' in dna_sequence or
                             (random.random() < 0.4 and gc_content > 0.55)),
            'idh_mutation': ('GGTCGT' in dna_sequence or
                            'CGTAGT' in dna_sequence or
                            (random.random() < 0.4 and gc_content < 0.45)),
            'mgmt_methylated': (gc_content > (0.6 - random.random()*0.1) or
                               (random.random() < 0.3 and 'CGCG' in dna_sequence))
        }

        results['dna_characteristics'] = {
            'gc_content': gc_content,
            'mutation_indicators': mutation_indicators
        }

        # Personalize treatments based on DNA characteristics with randomness
        if tumor_type == 'glioma':
            # IDH mutation treatment
            if mutation_indicators['idh_mutation']:
                # Choose randomly between treatment options
                idh_treatments = [
                    {
                        'name': 'Vorasidenib',
                        'smiles': 'CC1=C(C=C(C=C1)C2=CC=C(C=C2)C(=O)NC3=CC=CC=N3)N4C=NC(=N4)NC(=O)OC(C)(C)C',
                        'mechanism': 'IDH inhibitor',
                        'reason': 'IDH mutation detected'
                    },
                    {
                        'name': 'Ivosidenib',
                        'smiles': 'CCN1CCN(CC1)c1cc(c2c(c1)c(cn2CC(F)(F)C(F)(F)F)c1ccc(cc1)C(=O)O)C(F)(F)F',
                        'mechanism': 'IDH1 inhibitor',
                        'reason': 'IDH1 mutation signature identified'
                    }
                ]
                results['personalized_treatments'].append(random.choice(idh_treatments))

            # MGMT methylation treatment
            if mutation_indicators['mgmt_methylated']:
                # Random dose selection
                dose_options = ['High-dose', 'Escalating-dose', 'Extended-schedule']
                selected_dose = random.choice(dose_options)
                results['personalized_treatments'].append({
                    'name': f'{selected_dose} Temozolomide',
                    'smiles': 'CN1N=Nc2c(ncn2C)C1=O',
                    'mechanism': 'DNA alkylating agent (specialized regimen)',
                    'reason': 'MGMT methylation detected - increased sensitivity'
                })

            # TP53 mutation treatment - only sometimes suggested
            if mutation_indicators['tp53_mutation'] and random.random() > 0.4:
                tp53_treatments = [
                    {
                        'name': 'APR-246',
                        'smiles': 'CC(C)(C(=O)Nc1ccc(F)cc1)SC[C@@H]1CO1',
                        'mechanism': 'TP53 reactivator',
                        'reason': 'TP53 mutation detected'
                    },
                    {
                        'name': 'Adavosertib',
                        'smiles': 'CN1C(=O)C(=C(N)N)C(=O)N(C)c2nc(nc(c21)N1CCN(CC1)C(=O)NC1CC1)c1ccc(cc1)F',
                        'mechanism': 'WEE1 inhibitor for p53-deficient tumors',
                        'reason': 'TP53 mutation suggests synthetic lethality approach'
                    }
                ]
                results['personalized_treatments'].append(random.choice(tp53_treatments))

        elif tumor_type == 'meningioma':
            # Different treatment patterns based on GC content
            if gc_content > 0.55:
                results['personalized_treatments'].append({
                    'name': 'Mifepristone',
                    'smiles': 'CC#CC1(O)CCC2C3CCC4=CC(=O)CCC4=C3C(CC12C)c1ccc(cc1)N(C)C',
                    'mechanism': 'Progesterone receptor antagonist',
                    'reason': 'High GC content suggests hormone sensitivity'
                })
            elif gc_content < 0.4 and random.random() > 0.5:
                results['personalized_treatments'].append({
                    'name': 'Everolimus',
                    'smiles': 'COC(=O)C1CCC(C)C(O)(C(=O)C(=O)N2CCCCC2)C1',
                    'mechanism': 'mTOR inhibitor',
                    'reason': 'Low GC content profile suggests mTOR pathway activation'
                })
            # Random addition of experimental treatment
            if random.random() > 0.6:
                results['personalized_treatments'].append({
                    'name': 'Selumetinib',
                    'smiles': 'CN(C)c1cc2c(F)cc3c(c2cn1)N(C(C)CN1CCOCC1)C(=O)NC3=O',
                    'mechanism': 'MEK inhibitor',
                    'reason': 'Experimental treatment for recurrent meningioma'
                })

        elif tumor_type == 'pituitary':
            # Low GC content treatment
            if gc_content < 0.45:
                results['personalized_treatments'].append({
                    'name': 'Pasireotide',
                    'smiles': 'CC(C)C[C@@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)NC(=O)C(C)(C)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H]1CCCN1C(=O)[C@H](Cc1ccccc1)NC(=O)CNC(=O)CNC(=O)[C@@H](NC(=O)[C@@H](N)Cc1ccc(O)cc1)C(C)C)C(=O)N[C@@H](Cc1c[nH]c2ccccc12)C(=O)OH',
                    'mechanism': 'Multi-receptor somatostatin analog',
                    'reason': 'Low GC content indicating specific receptor profile'
                })
            # Check for specific repeat patterns in the DNA
            elif 'AGAG' in dna_sequence and random.random() > 0.4:
                results['personalized_treatments'].append({
                    'name': 'Pegvisomant',
                    'smiles': 'CCC(C)C1NC(=O)C2CSSCC(NC(=O)C(NC(=O)C(CCCNC(=N)N)NC1=O)CO)C(=O)N2',
                    'mechanism': 'Growth hormone receptor antagonist',
                    'reason': 'AG-repeat pattern suggesting growth hormone hypersecretion'
                })

        elif tumor_type == 'unknown_tumor':
            # For unknown tumors, suggest genetic profiling if DNA has unusual patterns
            unusual_patterns = ['ATATAT', 'CGCGCG', 'TATATATA', 'GCGCGCGC']
            has_unusual_pattern = any(pattern in dna_sequence for pattern in unusual_patterns)

            if has_unusual_pattern or random.random() > 0.7:
                results['personalized_treatments'].append({
                    'name': 'Comprehensive genomic profiling',
                    'smiles': None,
                    'mechanism': 'Next-generation sequencing analysis',
                    'reason': 'Complex DNA sequence suggests need for advanced molecular classification'
                })

    # If SmileBERT is available, use it to propose novel molecules (except for healthy patients)
    if smilebert_model and smilebert_tokenizer and dna_sequence and tumor_type != 'healthy':
        try:
            # Extract random fragment of DNA for more variation between runs
            start_pos = random.randint(0, max(0, len(dna_sequence)-40))
            dna_fragment = dna_sequence[start_pos:start_pos+30]

            # Use SmileBERT to encode the tumor DNA pattern
            inputs = smilebert_tokenizer(dna_fragment, return_tensors="pt")
            with torch.no_grad():
                outputs = smilebert_model(**inputs)

            # Simulate SmileBERT recommendation with more variety
            # Define multiple options for each tumor type
            tumor_treatments = {
                'glioma': [
                    {
                        'name': 'BERT-Glio-01',
                        'smiles': 'C1=CC=C2C(=C1)C(=CN2)CCN(C)C',
                        'mechanism': 'Novel tyrosine kinase inhibitor',
                        'confidence': f"{75 + random.randint(0, 15)}%"
                    },
                    {
                        'name': 'BERT-G7-IDH',
                        'smiles': 'Cc1cccc(NC(=O)c2ccc(cc2)CN2CCN(CC2)C(=O)N2CCC[C@H]2C(=O)NO)c1',
                        'mechanism': 'Dual IDH/HDAC inhibitor',
                        'confidence': f"{70 + random.randint(0, 20)}%"
                    },
                    {
                        'name': 'NeurInhibitor-X9',
                        'smiles': 'NC(=O)c1cccc(c1)C(=O)Nc1ccc2c(c1)N=C(N)N(C)C2=O',
                        'mechanism': 'Blood-brain barrier penetrant PI3K inhibitor',
                        'confidence': f"{65 + random.randint(0, 25)}%"
                    }
                ],
                'meningioma': [
                    {
                        'name': 'BERT-Menin-X4',
                        'smiles': 'CCC1=NN(C2=C1N=C(NC2=O)C3=CC=CC=C3)C4=CC=C(C=C4)S(=O)(=O)N',
                        'mechanism': 'Selective meningioma growth inhibitor',
                        'confidence': f"{70 + random.randint(0, 20)}%"
                    },
                    {
                        'name': 'Menin-T22',
                        'smiles': 'CC(C)c1ccc(cc1)S(=O)(=O)N1CCN(CC1)c1ccc(nc1)N=C(N)N',
                        'mechanism': 'Meningioma stem cell inhibitor',
                        'confidence': f"{65 + random.randint(0, 15)}%"
                    }
                ],
                'pituitary': [
                    {
                        'name': 'Pituistatix',
                        'smiles': 'CNC(=O)c1cc(cnc1N)c1ccc(cc1)NC(=O)Nc1cc(ccc1C)C(F)(F)F',
                        'mechanism': 'Receptor profile adaptive compound',
                        'confidence': f"{60 + random.randint(0, 30)}%"
                    },
                    {
                        'name': 'BERT-Neuro-Gen',
                        'smiles': 'CC(C)(C)NC(=O)[C@@H]1CCN(C[C@H]1N(C)S(=O)(=O)C)C(=O)[C@H](C(C)C)NC=O',
                        'mechanism': 'Brain-penetrant multi-kinase inhibitor',
                        'confidence': f"{65 + random.randint(0, 20)}%"
                    }
                ],
                'unknown_tumor': [
                    {
                        'name': 'Pan-Glio-RT9',
                        'smiles': 'CCN1CCN(CC1)c1cc2c(cc1F)c(cn2CC)C(=O)c1c(noc1C)C',
                        'mechanism': 'Pan-tumor brain penetrant compound',
                        'confidence': f"{50 + random.randint(0, 25)}%"
                    },
                    {
                        'name': 'BrainScan-X1',
                        'smiles': 'C=CC(=O)N1CCC(CC1)n1cc2c(c1)c(nc(n2)N)N1CCOCC1',
                        'mechanism': 'Blood brain barrier penetrant profiler',
                        'confidence': f"{55 + random.randint(0, 20)}%"
                    }
                ]
            }

            # Get treatment options for this tumor type, or use unknown_tumor as fallback
            treatment_options = tumor_treatments.get(tumor_type, tumor_treatments.get('unknown_tumor', []))

            # Randomly select a treatment option
            if treatment_options:
                novel_treatment = random.choice(treatment_options)
                results['ai_suggested_treatment'] = novel_treatment

        except Exception as e:
            logger.error(f"Error in SmileBERT treatment suggestion: {e}")
            results['ai_error'] = str(e)

    return results
# Function to visualize molecular structures of treatments
def visualize_treatments(treatments_data):
    valid_smiles = []
    labels = []

    # Collect valid SMILES from base treatments
    for treatment in treatments_data.get('base_treatments', []):
        if treatment['smiles']:
            mol = Chem.MolFromSmiles(treatment['smiles'])
            if mol:
                valid_smiles.append(mol)
                labels.append(f"{treatment['name']} ({treatment['mechanism']})")

    # Add personalized treatments
    for treatment in treatments_data.get('personalized_treatments', []):
        if treatment['smiles']:
            mol = Chem.MolFromSmiles(treatment['smiles'])
            if mol:
                valid_smiles.append(mol)
                labels.append(f"{treatment['name']} ({treatment['mechanism']})")

    # Add AI suggested treatment if available
    if 'ai_suggested_treatment' in treatments_data and treatments_data['ai_suggested_treatment']['smiles']:
        mol = Chem.MolFromSmiles(treatments_data['ai_suggested_treatment']['smiles'])
        if mol:
            valid_smiles.append(mol)
            labels.append(f"{treatments_data['ai_suggested_treatment']['name']} - AI Suggested")

    # If we have molecules to display
    if valid_smiles:
        # Calculate grid dimensions
        n_mols = len(valid_smiles)
        n_cols = min(3, n_mols)
        n_rows = (n_mols + n_cols - 1) // n_cols

        # Generate the image grid
        fig = plt.figure(figsize=(12, n_rows * 4))
        for i, (mol, label) in enumerate(zip(valid_smiles, labels)):
            ax = fig.add_subplot(n_rows, n_cols, i+1)
            img = Draw.MolToImage(mol, size=(300, 300))
            ax.imshow(img)
            ax.set_title(label)
            ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"treatments_for_{treatments_data['tumor_type']}.png")
        plt.close()
        logger.info(f"Treatment visualization saved as treatments_for_{treatments_data['tumor_type']}.png")
    else:
        logger.warning("No valid molecular structures to visualize")

# Function to select random samples from the DNA data
def select_random_samples(df, n=5, seed=42):
    """Select n random samples from the DataFrame, stratified by tumor type"""
    if df is None or df.empty:
        logger.error("No data available for sampling")
        return None

    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    # Try to get samples from each tumor type
    tumor_types = df['tumor_type'].unique()
    samples = []

    # Get at least one sample from each tumor type if possible
    for tumor_type in tumor_types:
        tumor_df = df[df['tumor_type'] == tumor_type]
        if len(tumor_df) > 0:
            samples.append(tumor_df.sample(1).iloc[0])

    # If we need more samples to reach n
    if len(samples) < n:
        # Convert samples to DataFrame
        selected_indices = [s.name for s in samples]
        remaining_df = df[~df.index.isin(selected_indices)]

        # Get additional random samples
        additional_samples = remaining_df.sample(min(n - len(samples), len(remaining_df)))
        samples.extend(additional_samples.to_dict('records'))

    # If we have more than n samples, trim the list
    if len(samples) > n:
        samples = samples[:n]

    logger.info(f"Selected {len(samples)} random samples for analysis")
    return pd.DataFrame(samples)

# Main function to process samples and generate treatment recommendations
def analyze_treatments_for_samples(samples_df):
    # Load SmileBERT model
    smilebert_model, smilebert_tokenizer = load_smilebert_model()

    # Process each sample
    results = []
    for _, sample in samples_df.iterrows():
        tumor_type = sample['tumor_type']
        dna_sequence = sample['dna_sequence']

        logger.info(f"Generating treatment recommendations for {tumor_type} tumor...")
        treatment_data = recommend_treatments(
            tumor_type,
            dna_sequence,
            smilebert_model,
            smilebert_tokenizer
        )

        # Visualize molecular structures
        visualize_treatments(treatment_data)

        results.append(treatment_data)

        # Print summary
        print(f"\nTreatment Summary for {tumor_type.upper()}:")
        print(f"DNA Sequence: {treatment_data['dna_sequence_preview']}")

        print("\nStandard Treatments:")
        for treatment in treatment_data['base_treatments']:
            print(f"- {treatment['name']} ({treatment['mechanism']})")

        if treatment_data['personalized_treatments']:
            print("\nPersonalized Treatments:")
            for treatment in treatment_data['personalized_treatments']:
                print(f"- {treatment['name']} ({treatment['mechanism']})")
                print(f"  Reason: {treatment.get('reason', 'DNA analysis')}")

        if 'ai_suggested_treatment' in treatment_data:
            treatment = treatment_data['ai_suggested_treatment']
            print("\nAI-Suggested Novel Treatment:")
            print(f"- {treatment['name']} ({treatment['mechanism']})")
            print(f"  Confidence: {treatment.get('confidence', 'N/A')}")

    return results

# Main execution block
if __name__ == "__main__":
    # Load the DNA sequence data
    df = load_dna_data()
    if df is None:
        logger.error("Failed to load DNA sequence data. Cannot continue.")
    else:
        logger.info(f"Loaded DNA data with tumor types: {df['tumor_type'].value_counts().to_dict()}")

        # Select a few samples from our dataset - one from each tumor type if possible
        sample_data = select_random_samples(df, n=5)  # Try to get one sample from each tumor type
        if sample_data is not None:
            logger.info(f"Selected samples with tumor types: {sample_data['tumor_type'].tolist()}")

            # Generate treatment recommendations
            treatment_results = analyze_treatments_for_samples(sample_data)

            # Create a summary table
            summary_rows = []
            for result in treatment_results:
                base_treatments = ", ".join([t['name'] for t in result['base_treatments']])
                personalized = ", ".join([t['name'] for t in result['personalized_treatments']]) if result['personalized_treatments'] else "None"
                ai_suggested = result.get('ai_suggested_treatment', {}).get('name', "None")

                summary_rows.append({
                    'Tumor Type': result['tumor_type'],
                    'Standard Treatments': base_treatments,
                    'Personalized Treatments': personalized,
                    'AI Suggestion': ai_suggested
                })

            summary_df = pd.DataFrame(summary_rows)
            print("\n===== TREATMENT SUMMARY TABLE =====")
            print(summary_df.to_string(index=False))

            # Save the summary table
            summary_df.to_csv("brain_tumor_treatment_summary.csv", index=False)
            logger.info("Treatment summary saved to brain_tumor_treatment_summary.csv")
        else:
            logger.error("Could not select any samples for analysis")


Treatment Summary for UNKNOWN_TUMOR:
DNA Sequence: ATACAAATACCCTCTTTAGGAGCAGTTGGTCCCGGGTGTGTAAGTCAATC...

Standard Treatments:
- Biopsy followed by targeted therapy (Molecular profiling and precision medicine)

AI-Suggested Novel Treatment:
- Pan-Glio-RT9 (Pan-tumor brain penetrant compound)
  Confidence: 71%

Treatment Summary for HEALTHY:
DNA Sequence: GAACTGTGTTTTGTTAGGACGCCCCGCTAAAGTGGCTGGTTGCTTTCATT...

Standard Treatments:
- No treatment needed (Monitoring only)

Treatment Summary for GLIOMA:
DNA Sequence: TAATGATAAAAGTGACGTGAGTTAGTGTCTAAATTTTCATTGACAACAGA...

Standard Treatments:
- Bevacizumab (VEGF inhibitor)

AI-Suggested Novel Treatment:
- NeurInhibitor-X9 (Blood-brain barrier penetrant PI3K inhibitor)
  Confidence: 77%

Treatment Summary for MENINGIOMA:
DNA Sequence: GAGCCTAGTATTCCAGACAAGAATCATAGTAGAACCCTGGGCGGCTATAG...

Standard Treatments:
- Surgery + Radiation (Physical removal and targeted radiation)

Personalized Treatments:
- Mifepristone (Progesterone receptor antago

In [40]:
def get_treatment_from_dna(dna_sequence, tumor_type=None):
    """
    Get treatment recommendations for a DNA sequence

    Parameters:
    -----------
    dna_sequence : str
        DNA sequence (A, T, G, C bases only)
    tumor_type : str, optional
        If known, specify 'glioma', 'meningioma', 'pituitary', 'healthy', or 'unknown_tumor'

    Returns:
    --------
    Dict containing treatment recommendations
    """
    import logging
    import time

    # Validate DNA sequence
    dna_sequence = dna_sequence.upper().strip()
    valid_bases = set('ATGC')
    if not all(base in valid_bases for base in dna_sequence):
        return {"error": "Invalid DNA sequence. Must contain only A, T, G, C bases."}

    if len(dna_sequence) < 20:
        return {"error": "DNA sequence too short (minimum 20 bases required)"}

    # If tumor type not specified, predict it from sequence
    if tumor_type is None:
        tumor_type = predict_tumor_type_from_dna(dna_sequence)
        print(f"Predicted tumor type: {tumor_type}")

    # Load SmileBERT model
    smilebert_model, smilebert_tokenizer = load_smilebert_model()

    # Get treatment recommendations
    treatments = recommend_treatments(
        tumor_type,
        dna_sequence,
        smilebert_model,
        smilebert_tokenizer
    )

    # Generate visualization
    visualize_treatments(treatments)
    print(f"Treatment visualization saved to: treatments_for_{tumor_type}.png")

    return treatments

def predict_tumor_type_from_dna(dna_sequence):
    """Predict tumor type based on DNA sequence patterns"""
    # Calculate GC content (proportion of G and C bases)
    gc_content = (dna_sequence.count('G') + dna_sequence.count('C')) / len(dna_sequence)

    # Look for tumor-specific patterns
    if gc_content < 0.4 or 'ATAT' in dna_sequence[:50]:
        return 'glioma'  # Low GC content typical for gliomas
    elif gc_content > 0.6 or 'GCGC' in dna_sequence[:50]:
        return 'meningioma'  # High GC content typical for meningiomas
    elif 'AGAG' in dna_sequence[:50]:
        return 'pituitary'  # Special pattern for pituitary tumors
    elif 0.45 < gc_content < 0.55:
        return 'healthy'  # Balanced GC content suggests healthy tissue
    else:
        return 'unknown_tumor'

# Example usage:
# Generate an example DNA sequence
example_dna = "ATGCATATATGCGCGCATAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAG"

# Get treatment recommendations
treatment_results = get_treatment_from_dna(example_dna)

# Display key treatment information
print("\n===== TREATMENT RECOMMENDATIONS =====")
print(f"Tumor type: {treatment_results['tumor_type']}")
print(f"DNA sequence: {treatment_results['dna_sequence_preview']}")

print("\nStandard treatments:")
for t in treatment_results['base_treatments']:
    print(f"- {t['name']} ({t['mechanism']})")

if treatment_results.get('personalized_treatments'):
    print("\nPersonalized treatments:")
    for t in treatment_results['personalized_treatments']:
        print(f"- {t['name']} ({t['mechanism']})")
        print(f"  Reason: {t.get('reason', 'DNA analysis')}")

if 'ai_suggested_treatment' in treatment_results:
    t = treatment_results['ai_suggested_treatment']
    print(f"\nAI-suggested treatment ({t.get('confidence', 'N/A')} confidence):")
    print(f"- {t['name']} ({t['mechanism']})")

Predicted tumor type: glioma
Treatment visualization saved to: treatments_for_glioma.png

===== TREATMENT RECOMMENDATIONS =====
Tumor type: glioma
DNA sequence: ATGCATATATGCGCGCATAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCT...

Standard treatments:
- Temozolomide (DNA alkylating agent)
- Bevacizumab (VEGF inhibitor)

Personalized treatments:
- High-dose Temozolomide (DNA alkylating agent (specialized regimen))
  Reason: MGMT methylation detected - increased sensitivity

AI-suggested treatment (80% confidence):
- BERT-Glio-01 (Novel tyrosine kinase inhibitor)


In [41]:
!pip install flask
!pip install pyngrok
!ngrok config add-authtoken 2u1MKS8BtGwqwMV9zVTn4FCYoX1_36wv8gX89V2h3Jfm2VWHZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
